In [2]:
import cobra
from cobra.io import load_model
import escher
from escher import Builder
from time import sleep
import pandas

In [ ]:
#Run FBA, see growth rate under normal import circumstances
solution = model.optimize()
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')

In [ ]:
#what if we decrease 'amount' of glucose in medium (i.e. decrease import rate of glucose? what happens to growth rate?)
model.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000

model.reactions.get_by_id("EX_glc__D_e").lower_bound = -10
solution = model.optimize()
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')


In [ ]:
#convert fluxes to df
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass.csv')

In [4]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [3]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='iJO1366',
)
builder

Builder()

In [ ]:
#simulate deletions
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
with model:
    model.reactions.PGI.knock_out()
    print('PGI knocked out: ', model.optimize())
    solution = model.optimize()
    print('Growth rate: ' + str(solution.objective_value) + ' 1/h')
    df = pandas.DataFrame.from_dict([solution.fluxes]).T
    df.to_csv('FBA_max_biomas1.csv')  

In [ ]:
#add heterologous reactions for genes crtE, crtB, crtI
